In [16]:
import requests
import pandas as pd
import numpy as np
import math
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from IPython.display import Markdown as md
import os

# Initialize Bokeh for charts
output_notebook()

# Pull latest data
url = "https://covidtracking.com/api/states"
r = requests.get(url)

# Convert data to json
j = r.json()

# Load data into dataframe
sars = pd.DataFrame(j)

# COLUMN REFERENCE
# stats: state, positive, negative, pending, hospitalized, death, lastUpdateEt, totalTestResults, grade, score

# Drop columns not currently being used
drop_columns = ["positiveScore",
                "negativeScore",
                "negativeRegularScore",
                "commercialScore",
                "total",
                "checkTimeEt",
                "fips",
                "dateModified",
                "dateChecked",
                "notes",
                "hash",
               ]
sars.drop(columns=drop_columns, inplace=True)

# Drop states
drop_states = ["PR",
               "AS",
               "GU",
               "MP",
               "VI",
              ]
for state in drop_states:
    sars.drop(sars[sars.state == state].index, inplace=True)

# Fill any missing data with zeroes
sars['pending'].fillna(0, inplace=True)
sars['hospitalized'].fillna(0, inplace=True)
sars['death'].fillna(0, inplace=True)
sars['negative'].fillna(0, inplace=True)
sars['score'].fillna(0, inplace=True)


# Load State Population file
with open('state_populations.json') as data:
    state_pop = pd.read_json(data)

# Merge state population data with covid data into new covid dataframe
covid = pd.merge(left=sars, right=state_pop[['state_code','state_name','population']], how='left', left_on='state', right_on='state_code')

# Create new columns in covid dataframe
covid['infectionRate'] = covid['positive'].div(covid['population'], fill_value=0)
covid['positiveRate'] = covid['positive'].div(covid['totalTestResults'].sub(covid['pending']), fill_value=0)
covid['negativeRate'] = covid['negative'].div(covid['totalTestResults'].sub(covid['pending']), fill_value=0)
covid['hospitalRate'] = covid['hospitalized'].div(covid['positive'], fill_value=0)
covid['deathRate'] = covid['death'].div(covid['positive'], fill_value=0)
covid['testRate'] = covid['totalTestResults'].div(covid['population'], fill_value=0)

# Reorder dataframe
covid = covid[['state_name',
               'state',
               'population',
               'positive',
               'negative',
               'pending',
               'totalTestResults',
               'infectionRate',
               'positiveRate',
               'negativeRate',
               'hospitalized',
               'hospitalRate',
               'death',
               'deathRate',
               'testRate',
             ]]

# Create index list for for bokeh graphs
index = covid['state_name'].to_list()

# Set new index for covid dataframe
covid.set_index('state_name', inplace=True)

# Sum US Statistics from covid dataframe 
total_population = int(covid['population'].sum())
total_positives = int(covid['positive'].sum())
total_negatives = int(covid['negative'].sum())
total_pendings = int(covid['pending'].sum())
total_hospitalized = int(covid['hospitalized'].sum())
total_test_results = int(covid['totalTestResults'].sum())
total_deaths = int(covid['death'].sum())

# Calculate rates for US Statistics
infection_rate = total_positives / total_population
positive_rate = total_positives / (total_test_results - total_pendings)
negative_rate = total_negatives / (total_test_results - total_pendings)
pending_rate = total_pendings / total_test_results
hospitalization_rate = total_hospitalized / total_positives
death_rate = total_deaths / total_positives
test_rate = total_test_results / total_population

# Create tracking data for various metrics to see how each state is performing
covid['expectedPositives'] = covid['population'].mul(infection_rate)
covid['positiveDelta'] = covid['positive'].sub(covid['expectedPositives'])
covid['positiveDeltaRate'] = covid['positiveDelta'].div(covid['expectedPositives'], fill_value=0)

covid['expectedNegatives'] = covid['totalTestResults'].mul(negative_rate)
covid['negativeDelta'] = covid['negative'].sub(covid['expectedNegatives'])
covid['negativeDeltaRate'] = covid['negativeDelta'].div(covid['expectedNegatives'], fill_value=0)

covid['expectedHospitalizations'] = covid['positive'].mul(hospitalization_rate)
covid['hospitalDelta'] = covid['hospitalized'].sub(covid['expectedHospitalizations'])
covid['hospitalDeltaRate'] = covid['hospitalDelta'].div(covid['expectedHospitalizations'], fill_value=0)

covid['expectedDeaths'] = covid['positive'].mul(death_rate)
covid['deathDelta'] = covid['death'].sub(covid['expectedDeaths'])
covid['deathDeltaRate'] = covid['deathDelta'].div(covid['expectedDeaths'], fill_value=0)

covid['expectedTests'] = covid['population'].mul(test_rate)
covid['testDelta'] = covid['totalTestResults'].sub(covid['expectedTests'])
covid['testDeltaRate'] = covid['testDelta'].div(covid['expectedTests'], fill_value=0)

covid['expectedPending'] = covid['totalTestResults'].mul(pending_rate)
covid['pendingDelta'] = covid['pending'].sub(covid['expectedPending'])
covid['pendingDeltaRate'] = covid['pendingDelta'].div(covid['expectedPending'], fill_value=0)

covid['populationShare'] = covid['population'].div(total_population, fill_value=0)
covid['positiveShare'] = covid['positive'].div(total_positives, fill_value=0)
covid['negativeShare'] = covid['negative'].div(total_negatives, fill_value=0)
covid['pendingShare'] = covid['pending'].div(total_pendings, fill_value=0)
covid['hospitalShare'] = covid['hospitalized'].div(total_hospitalized, fill_value=0)
covid['deathShare'] = covid['death'].div(total_deaths, fill_value=0)
covid['testShare'] = covid['totalTestResults'].div(total_test_results, fill_value=0)

Loading BokehJS ...

In [17]:
# Define which states to track in the markdown sections below.
# Standard Capitalization for each state name in the comma-separated list
states_to_watch = ["Tennessee", "Kentucky", "Indiana", "New York"]

## SARS-CoV-2 / COVID-19 Testing

In [18]:
stat_tuples_array = [('Positive Population %', "infectionRate", infection_rate, 0),
                     ('Positive Test Rate', "positiveRate", positive_rate, 0),
                     ('Negative Test Rate', "negativeRate", negative_rate, 0),
                     ('Expected Positives', "expectedPositives", '-----', 1),
                     ('Actual Positives', "positive", total_positives, 1),
                     ("Positive Delta", "positiveDelta", '-----', 1),
                     ('Delta %', "positiveDeltaRate", '-----', 0),
                     ('Expected Tests', "expectedTests", '-----', 1),
                     ('Actual Tests', "totalTestResults", total_test_results, 1),
                     ('Test Delta', "testDelta", '-----', 1),
                     ('Delta %', "testDeltaRate", '-----', 0),
                     ('Expected Negatives', "expectedNegatives", '-----', 1),
                     ('Actual Negatives', "negative", total_negatives, 1),
                     ('Negative Delta', "negativeDelta", '-----', 1),
                     ('Delta %', "negativeDeltaRate", '-----', 0),
                     ('Expected Pending', "expectedPending", '-----', 1),
                     ('Actual Pending', "pending", total_pendings, 1),
                     ('Pending Delta', "pendingDelta", '-----', 1),
                     ('Delta %', "pendingDeltaRate", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `Kentucky` | `Indiana` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: |
| Positive Population % | `0.0199%` | `0.0088%` | `0.0183%` | `0.2691%` | `0.0366%` |
| Positive Test Rate | `7.4872%` | `7.1106%` | `14.6545%` | `33.5514%` | `17.4312%` |
| Negative Test Rate | `92.5128%` | `92.8894%` | `85.3455%` | `66.4486%` | `91.8893%` |
| Expected Positives | `2,525` | `1,647` | `2,469` | `7,116` | `-----` |
| Actual Positives | `1,373` | `394` | `1,232` | `52,318` | `121,288` |
| Positive Delta | `-1,152` | `-1,253` | `-1,237` | `45,201` | `-----` |
| Delta % | `-45.6245%` | `-76.0810%` | `-50.1075%` | `635.1455%` | `-----` |
| Expected Tests | `15,835` | `10,330` | `15,486` | `44,632` | `-----` |
| Actual Tests | `18,338` | `5,541` | `8,407` | `155,934` | `760,662` |
| Test Delta | `2,502` | `-4,789` | `-7,079` | `111,301` | `-----` |
| Delta % | `15.8005%` | `-46.3635%` | `-45.7136%` | `249.3728%` | `-----` |
| Expected Negatives | `16,850` | `5,091` | `7,725` | `143,286` | `-----` |
| Actual Negatives | `16,965` | `5,147` | `7,175` | `103,616` | `639,374` |
| Negative Delta | `114` | `55` | `-550` | `-39,670` | `-----` |
| Delta % | `0.6786%` | `1.0883%` | `-7.1213%` | `-27.6862%` | `-----` |
| Expected Pending | `1,563` | `472` | `716` | `13,294` | `-----` |
| Actual Pending | `0` | `0` | `0` | `0` | `64,853` |
| Pending Delta | `-1,563` | `-472` | `-716` | `-13,294` | `-----` |
| Delta % | `-100.0000%` | `-100.0000%` | `-100.0000%` | `-100.0000%` | `-----` |


## Hospitalizations

In [19]:
stat_tuples_array = [('Hospitalization %', "hospitalRate", hospitalization_rate, 0),
                     ('Actual Positives', "positive", total_positives, 1),
                     ('Expected Hospitalizations', "expectedHospitalizations", '-----', 1),
                     ('Actual Hospitalizations', "hospitalized", total_hospitalized, 1),
                     ('Hospitalized Delta', "hospitalDelta", '-----', 1),
                     ('Delta %', "hospitalDeltaRate", '-----', 0),
                     ('Hospital Share', "hospitalShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `Kentucky` | `Indiana` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: |
| Hospitalization % | `8.5943%` | `0.0000%` | `0.0000%` | `19.2171%` | `13.9898%` |
| Actual Positives | `1,373` | `394` | `1,232` | `52,318` | `121,288` |
| Expected Hospitalizations | `192` | `55` | `172` | `7,319` | `-----` |
| Actual Hospitalizations | `118` | `0` | `0` | `10,054` | `16,968` |
| Hospitalized Delta | `-74` | `-55` | `-172` | `2,734` | `-----` |
| Delta % | `-38.5674%` | `-100.0000%` | `-100.0000%` | `37.3646%` | `-----` |
| Hospital Share | `0.6954%` | `0.0000%` | `0.0000%` | `59.2527%` | `-----` |


## Mortality

In [20]:
stat_tuples_array = [('Death Rate', "deathRate", death_rate, 0),
                     ('Actual Positives', "positive", total_positives, 1),
                     ('Actual Hospitalizations', "hospitalized", total_hospitalized, 1),
                     ('Hospitalization %', "hospitalRate", hospitalization_rate, 0),
                     ('Expected Deaths', "expectedDeaths", '-----', 1),
                     ('Actual Deaths', "death", total_deaths, 1),
                     ('Deaths Delta', "deathDelta", '-----', 1),
                     ('Delta %', "deathDeltaRate", '-----', 0),
                     ('Death Share', "deathShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `Kentucky` | `Indiana` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: |
| Death Rate | `0.4370%` | `2.0305%` | `2.5162%` | `1.3915%` | `1.6828%` |
| Actual Positives | `1,373` | `394` | `1,232` | `52,318` | `121,288` |
| Actual Hospitalizations | `118` | `0` | `0` | `10,054` | `16,968` |
| Hospitalization % | `8.5943%` | `0.0000%` | `0.0000%` | `19.2171%` | `13.9898%` |
| Expected Deaths | `23` | `6` | `20` | `880` | `-----` |
| Actual Deaths | `6` | `8` | `31` | `728` | `2,041` |
| Deaths Delta | `-17` | `1` | `10` | `-152` | `-----` |
| Delta % | `-74.0310%` | `20.6615%` | `49.5291%` | `-17.3096%` | `-----` |
| Death Share | `0.2940%` | `0.3920%` | `1.5189%` | `35.6688%` | `-----` |


## Percent Share of Total US

In [21]:
stat_tuples_array = [('Population Share', "populationShare", '-----', 0),
                     ('Positive Share', "positiveShare", '-----', 0),
                     ('Hospital Share', "hospitalShare", '-----', 0),
                     ('Death Share', "deathShare", '-----', 0),
                     ('Test Share', "testShare", '-----', 0),
                     ('Negative Share', "negativeShare", '-----', 0),
                     ('Pending Share', "pendingShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `Kentucky` | `Indiana` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: |
| Population Share | `2.0819%` | `1.3581%` | `2.0359%` | `5.8676%` | `-----` |
| Positive Share | `1.1320%` | `0.3248%` | `1.0158%` | `43.1353%` | `-----` |
| Hospital Share | `0.6954%` | `0.0000%` | `0.0000%` | `59.2527%` | `-----` |
| Death Share | `0.2940%` | `0.3920%` | `1.5189%` | `35.6688%` | `-----` |
| Test Share | `2.4108%` | `0.7284%` | `1.1052%` | `20.4998%` | `-----` |
| Negative Share | `2.6534%` | `0.8050%` | `1.1222%` | `16.2059%` | `-----` |
| Pending Share | `0.0000%` | `0.0000%` | `0.0000%` | `0.0000%` | `-----` |


## Charts

In [22]:
source = ColumnDataSource(data=covid)
plot_height = 600
plot_width = 1000
width = 0.5

title_p = "COVID Confirmed: {:,} - Death Rate: {:.4%} - Hospitalization Rate: {:.4%}".format(total_positives, death_rate, hospitalization_rate)
title_q = "COVID Deaths: {:,} - Death Rate: {:.4%}".format(total_deaths, death_rate)
title_z = "COVID Hospitalized: {:,} - Hospitalization Rate: {:.4%}".format(total_hospitalized, hospitalization_rate)
title_x = "COVID US National Infection Rate: {:.4%}".format(infection_rate)
title_y = "COVID US Confirmed / Expected Delta"
title_m = "COVID US Percentage Share of Confirmed Cases by State"
title_n = "COVID US Percentage Share of Deaths by State"
title_o = "COVID US Percentage Share of Hospitalizations by State" # FIX O

pTOOLTIPS = [
    ("State", "@state_name"),
    ("Cases", "@positive{0,0}"),
]
qTOOLTIPS = [
    ("State", "@state_name"),
    ("Deaths", "@death{0,0}"),
]
zTOOLTIPS = [
    ("State", "@state_name"),
    ("Hospitalized", "@hospitalized{0,0}"),
]
xTOOLTIPS = [
    ("State", "@state_name"),
    ("Inf. Rate", "@infectionRate{0.0000%}"),
]
yTOOLTIPS = [
    ("State", "@state_name"),
    ("Positive Delta", "@positiveDelta{0,0}"),
]
mTOOLTIPS = [
    ("State", "@state_name"),
    ("Positive Share", "@positiveShare{0.0000%}"),
]
nTOOLTIPS = [
    ("State", "@state_name"),
    ("Death Share", "@deathShare{0.0000%}"),
]
oTOOLTIPS = [
    ("State", "@state_name"),
    ("Hosp. Share", "@hospitalShare{0.0000%}"),
]


p = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_p, toolbar_location="above", tooltips=pTOOLTIPS)
p.vbar(x='state_name', top='positive', width=width, source=source, line_color='white')
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = covid['positive'].max() + covid['positive'].std()
p.xaxis.major_label_orientation = math.pi / 4
p.yaxis.axis_label = "Confirmed Cases"
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

q = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_q, toolbar_location="above", tooltips=qTOOLTIPS)
q.vbar(x='state_name', top='death', width=width, source=source, line_color='white')
q.xgrid.grid_line_color = None
q.y_range.start = 0
q.y_range.end = covid['death'].max() + covid['death'].std()
q.xaxis.major_label_orientation = math.pi / 4
q.yaxis.axis_label = "Confirmed Deaths"
q.yaxis.formatter = NumeralTickFormatter(format="0,0")

# FIX Z
z = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_z, toolbar_location="above", tooltips=zTOOLTIPS)
z.vbar(x='state_name', top='hospitalized', width=width, source=source, line_color='white')
z.xgrid.grid_line_color = None
z.y_range.start = 0
z.y_range.end = covid['hospitalized'].max() + covid['hospitalized'].std()
z.xaxis.major_label_orientation = math.pi / 4
z.yaxis.axis_label = "Confirmed Hospitalizations"
z.yaxis.formatter = NumeralTickFormatter(format="0,0")

x = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_x, toolbar_location="above", tooltips=xTOOLTIPS)
x.vbar(x='state_name', top='infectionRate', width=width, source=source)
x.xgrid.grid_line_color = None
x.y_range.start = 0
x.y_range.end = covid['infectionRate'].max() + covid['infectionRate'].std()
x.xaxis.major_label_orientation = math.pi / 4
x.yaxis.axis_label = "Confirmed Cases as % of State Population"
x.yaxis.formatter = NumeralTickFormatter(format="0.0000%")

y = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_y, toolbar_location="above", tooltips=yTOOLTIPS)
y.vbar(x='state_name', top='positiveDelta', width=width, source=source)
y.xgrid.grid_line_color = None
y.y_range.start = covid['positiveDelta'].min() - covid['positiveDelta'].std()
y.y_range.end = covid['positiveDelta'].max() + covid['positiveDelta'].std()
y.xaxis.major_label_orientation = math.pi / 4
y.yaxis.axis_label = "Actual Covid Cases - Expected Covid Cases"
y.yaxis.formatter = NumeralTickFormatter(format="0,0")

m = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_m, toolbar_location="above", tooltips=mTOOLTIPS)
m.vbar(x='state_name', top='positiveShare', width=width, source=source)
m.xgrid.grid_line_color = None
m.y_range.start = 0
m.y_range.end = covid['positiveShare'].max() + covid['positiveShare'].std()
m.xaxis.major_label_orientation = math.pi / 4
m.yaxis.axis_label = "Confirmed Cases as % of Total US Cases"
m.yaxis.formatter = NumeralTickFormatter(format="0.0000%")

n = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_n, toolbar_location="above", tooltips=nTOOLTIPS)
n.vbar(x='state_name', top='deathShare', width=width, source=source)
n.xgrid.grid_line_color = None
n.y_range.start = 0
n.y_range.end = covid['deathShare'].max() + covid['deathShare'].std()
n.xaxis.major_label_orientation = math.pi / 4
n.yaxis.axis_label = "Confirmed Deaths as % of Total US Covid Deaths"
n.yaxis.formatter = NumeralTickFormatter(format="0.0000%")


# FIX O
o = figure(x_range=index, plot_height=plot_height, plot_width=plot_width, title=title_o, toolbar_location="above", tooltips=oTOOLTIPS)
o.vbar(x='state_name', top='hospitalShare', width=width, source=source)
o.xgrid.grid_line_color = None
o.y_range.start = 0
o.y_range.end = covid['hospitalShare'].max() + covid['hospitalShare'].std()
o.xaxis.major_label_orientation = math.pi / 4
o.yaxis.axis_label = "Confirmed Recoveries as % of Total US Recoveries"
o.yaxis.formatter = NumeralTickFormatter(format="0.0000%")

show(x)
show(y)
show(m)
show(n)
show(p)
show(q)
show(z)
show(o)